In [261]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated, Literal
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
import operator

In [262]:
load_dotenv()

True

In [263]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation")
model = ChatHuggingFace(llm=llm)

In [264]:
class SentimentSchema(BaseModel):

    sentiment : Literal['positive', 'negative'] = Field(description="The sentiment of the review")
    

In [265]:
class DiagnosisSchema(BaseModel):

    issue_type: Literal['billing', 'technical', 'account', 'other'] = Field(description="The type of issue the customer is facing")
    tone: Literal['angry', 'sad', 'neutral', 'happy'] = Field(description="The tone of the customer")
    urgency: Literal['low', 'medium', 'high'] = Field(description="The urgency of the issue")

In [266]:
parser2=PydanticOutputParser(pydantic_object=DiagnosisSchema)

In [267]:
parser= PydanticOutputParser(pydantic_object=SentimentSchema)

In [268]:
prompt2= PromptTemplate(
    template=(
        "Analyze the following customer review and provide a diagnosis in JSON format.\n\n"
        "Review: {review}\n\n"
        "Return your answer in the following JSON format:\n{response_format}"
    ),
    input_variables=['review'],
    partial_variables={'response_format': parser2.get_format_instructions()}
)

In [269]:
structured_model2= prompt2 | model | parser2

In [270]:
prompt = PromptTemplate(
    template=(
        "Classify the sentiment of this review as positive or negative.\n\n"
        "Review: {review}\n\n"
        "Return your answer in the following JSON format:\n{response_format}"
    ),
    input_variables=['review'],
    partial_variables={'response_format': parser.get_format_instructions()}
)


In [271]:
structured_model= prompt | model | parser

In [272]:
class ReviewState(TypedDict):

    review: str
    sentiment: Literal['positive', 'negative']
    diagnosis: dict
    response: str

In [273]:
def find_sentiment(state: ReviewState):
    review = state['review']
    prompt = f"find the sentiment of this review \n {state['review']}"
    sentiment = structured_model.invoke(prompt).sentiment
    return {'sentiment': sentiment}   


In [274]:
def check_condition(state: ReviewState) -> Literal['positive_response', 'run_diagnosis']:

    if state['sentiment']=='positive':
        return 'positive_response'
    else:
        return 'run_diagnosis'

In [275]:
def positive_response(state: ReviewState):

    prompt=f"write a positive response to this review: {state['review']}"
    response=model.invoke(prompt).content
    return {'response': response}

def run_diagnosis(state: ReviewState):

    prompt=f"""Diagnose this negative review:\n\n{state['review']}\n"
    "Return issue_type, tone, and urgency.
"""
    response=structured_model2.invoke(prompt)
    return {'diagnosis': response.model_dump()}

def negative_response(state: ReviewState):

    prompt=f"write a sympathetic response to this review: {state['review']}. The issue type is {state['diagnosis']['issue_type']}, the tone is {state['diagnosis']['tone']}, and the urgency is {state['diagnosis']['urgency']}."
    response=model.invoke(prompt).content
    return {'response': response}

In [276]:
graph=StateGraph(ReviewState)

graph.add_node('find_sentiment', find_sentiment)
graph.add_node('run_diagnosis', run_diagnosis)
graph.add_node('positive_response', positive_response)
graph.add_node('negative_response', negative_response)

graph.add_edge(START, 'find_sentiment')
graph.add_conditional_edges('find_sentiment', check_condition)
graph.add_edge('run_diagnosis', 'negative_response')
graph.add_edge('negative_response', END)
graph.add_edge('positive_response', END)


workflow=graph.compile()


In [280]:
initial_state={
    'review': 'this is the worst product I have ever bought. It broke after one use and the customer service was terrible. I want a refund!'}

workflow.invoke(initial_state)

{'review': 'this is the worst product I have ever bought. It broke after one use and the customer service was terrible. I want a refund!',
 'sentiment': 'negative',
 'diagnosis': {'issue_type': 'technical', 'tone': 'angry', 'urgency': 'high'},
 'response': "I'm really sorry to hear that you had such a disappointing experience with our product. I understand your frustration, and I apologize for the inconvenience this has caused you. Let me see what I can do to help.\n\nFirst, I'd like to look into the technical issue you mentioned. Can you please describe in more detail what happened when the product broke after one use? This will help me understand the problem and provide a suitable solution.\n\nAs for the customer service, I would love to hear more about your interaction with us so I can work on improving it in the future.\n\nIt's not our intention to let our customers down, and I am committed to making this right for you. A refund is definitely an option if we can't resolve the techn